In [1]:
!pip install torch_geometric pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.8.0+cu126.html

Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00


In [2]:
import torch
import optuna
import torch.nn as nn
import torch.nn.functional as F
import json

from torch_geometric.loader import DataLoader
from torch_geometric import seed_everything
from torch_geometric.nn import GINConv, global_add_pool
from sklearn import metrics

In [3]:
class GIN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super().__init__()

        self.dropout = dropout
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for _ in range(num_layers - 1):
            self.convs.append(
                GINConv(nn.Sequential(
                    nn.Linear(input_dim, 2 * hidden_dim),
                    nn.BatchNorm1d(2 * hidden_dim),
                    nn.ReLU(),
                    nn.Linear(2 * hidden_dim, hidden_dim),
                ))
            )
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))
            input_dim = hidden_dim

        self.lin1 = nn.Linear(hidden_dim, hidden_dim)
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim)
        self.classifier = nn.Linear(hidden_dim, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        for conv, batch_norm in zip(self.convs, self.batch_norms):
            x = F.relu(batch_norm(conv(x, edge_index)))
            x = F.dropout(x, self.dropout, training=self.training)
        x = global_add_pool(x, batch)
        x = F.relu(self.batch_norm1(self.lin1(x)))
        x = F.dropout(x, self.dropout, training=self.training)
        return self.classifier(x).view(-1)

In [4]:
train_dataset = torch.load('/kaggle/input/graphs-with-automorphisms/train_dataset.pt',weights_only=False)
val_dataset = torch.load('/kaggle/input/graphs-with-automorphisms/val_dataset.pt',weights_only=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def objective(trial: optuna.Trial):
    global model, optimizer, criterion, scheduler, train_loader, val_loader

    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    num_layers = trial.suggest_int('num_layers', 3, 6)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    model = GIN(3, hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3
    )

    n_epochs = 50
    best_val_acc = 0.0
    patience = 10
    patience_counter = 0

    trial_history = []
    
    for epoch in range(1, n_epochs + 1):
        train_loss = train()
        train_acc, train_f1 = test(train_loader)
        val_acc, val_f1 = test(val_loader)
        
        scheduler.step(val_acc)

        trial_history.append({
            'trial': trial.number,
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'lr': optimizer.param_groups[0]['lr']
        })
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1
        
        trial.report(val_acc, epoch)
        
        if trial.should_prune():
            raise optuna.TrialPruned()
        
        if patience_counter >= patience:
            print(f"Trial {trial.number}: Early stopping at epoch {epoch}")
            break
        
        if epoch % 10 == 0:
            print(f"Trial {trial.number} | Epoch {epoch:02d} | "
                  f"Train Loss: {train_loss:.4f} | "
                  f"Val Acc: {val_acc:.4f}")
    
    return best_val_acc

In [6]:
def train():
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        out = model(data)
        loss = criterion(out, data.y.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs
        
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    predictions = []
    labels = []

    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = (out > 0).float()
        predictions.append(pred.cpu())
        labels.append(data.y.cpu())

    accuracy = metrics.accuracy_score(torch.cat(labels), torch.cat(predictions))
    f1 = metrics.f1_score(torch.cat(labels), torch.cat(predictions))

    return accuracy, f1

In [7]:
study = optuna.create_study(
    direction="maximize", 
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10),
    study_name="GIN for partial automorphism extension problem")

study.optimize(objective, n_trials=100)

trials_df = study.trials_dataframe()
trials_df.to_csv("/kaggle/working/optuna_trials_summary.csv", index=False)

trial = study.best_trial
print(f"  Value (Val Acc): {trial.value:.4f}")
print("\n  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

best_params = study.best_params
config_path = f"/kaggle/working/best_config.json"
with open(config_path, "w") as f:
    json.dump(best_params, f, indent=4)

[I 2026-02-14 23:11:26,632] A new study created in memory with name: GIN for partial automorphism extension problem


Trial 0 | Epoch 10 | Train Loss: 0.5163 | Val Acc: 0.7490
Trial 0 | Epoch 20 | Train Loss: 0.4712 | Val Acc: 0.7750
Trial 0 | Epoch 30 | Train Loss: 0.4445 | Val Acc: 0.7888
Trial 0 | Epoch 40 | Train Loss: 0.4105 | Val Acc: 0.8021


[I 2026-02-14 23:22:46,649] Trial 0 finished with value: 0.8091506849315069 and parameters: {'hidden_dim': 512, 'num_layers': 3, 'dropout': 0.30673257824787586, 'lr': 0.0008858959635662728, 'weight_decay': 1.6061949156274486e-05, 'batch_size': 64}. Best is trial 0 with value: 0.8091506849315069.


Trial 0 | Epoch 50 | Train Loss: 0.3896 | Val Acc: 0.8092
Trial 1 | Epoch 10 | Train Loss: 0.5007 | Val Acc: 0.7510
Trial 1 | Epoch 20 | Train Loss: 0.4560 | Val Acc: 0.7730
Trial 1 | Epoch 30 | Train Loss: 0.4227 | Val Acc: 0.7876
Trial 1 | Epoch 40 | Train Loss: 0.4005 | Val Acc: 0.7902


[I 2026-02-14 23:30:38,441] Trial 1 finished with value: 0.803068493150685 and parameters: {'hidden_dim': 256, 'num_layers': 4, 'dropout': 0.15748470515581697, 'lr': 0.0007334864828716476, 'weight_decay': 0.00018554687089747545, 'batch_size': 128}. Best is trial 0 with value: 0.8091506849315069.


...

Trial 98 | Epoch 10 | Train Loss: 0.4884 | Val Acc: 0.7574


[I 2026-02-15 07:23:15,028] Trial 98 pruned. 


Trial 99 | Epoch 10 | Train Loss: 0.4921 | Val Acc: 0.7614


[I 2026-02-15 07:25:41,573] Trial 99 pruned. 


  Value (Val Acc): 0.8107

  Params: 
    hidden_dim: 256
    num_layers: 3
    dropout: 0.04821922755593036
    lr: 0.0008007016085176578
    weight_decay: 1.5408221478908417e-05
    batch_size: 64
